# Simple Price Yield

This notebook demonstrates how to access and use the functionalities of **Price Yield** which are part of our **Yiedbook** module within LSEG Financial Analytics SDK. The notebook outlines the most simple method of using performing Price Yield, where only the required set of arguments are present.

The main goal of this notebook is to explore and display different approaches in retrieval of single instrument simple Price Yield calcuation by using any of 4 available methods:
1. Synchronous POST
2. Synchronous GET
3. Asynchronous POST
4. Asynchronous GET

## Imports

Import the following necessary modules:

- *request_py_calculation_sync_by_id* - main Synchronous GET method
- *request_py_calculation_sync* - main Synchronous POST method
- *request_py_calculation_async_by_id* - main Asynchronous GET method
- *request_py_calculation_async* - main Asynchronous POST method
- *get_result* - method for obtaining results of Async method executions
- *json* - for display purposes 
- *time* - for response delay handling

In [1]:
from lseg_analytics.yield_book_rest import (
    request_py_calculation_sync_by_id,
    request_py_calculation_sync,
    request_py_calculation_async_by_id,    
    request_py_calculation_async,
    get_result,
    PyCalcGlobalSettings,
    PyCalcInput,
    CurveTypeAndCurrency,
    Volatility
)

from datetime import date

import json as js

import time

## Data Preparation

Prepare the 'input' argument which contains an identifier and price level. It will be consumed by multiple methods.

In [2]:
# Select an ISIN or CUSIP ID of the instrument
identifier="31398GY86"

# Set a pricing level for the calculation
price_level = 99


Outlined below are all the available properties the user can set for the POST request. For arguments that are not required, default values will be used in their absence.

In [3]:
global_settings = PyCalcGlobalSettings(
            pricing_date=date(2025, 1, 1)
        )

input = [
            PyCalcInput(
                identifier=identifier,
                level=price_level,   
                curve=CurveTypeAndCurrency(
                    curve_type="GVT",
                    currency="USD",
                    retrieve_curve=False,
                    snapshot="EOD",
                ),                  
                volatility=Volatility(
                    type="LMMDD"
                ),               
            )
        ]

## Synchronous POST Data retrieval

Once the input arguments are ready, they can be passed through the main method for data retrieval.

In [4]:
# Request bond PY with sync POST
py_sync_post_response = request_py_calculation_sync(
            global_settings=global_settings,
            input=input,
            keywords=["yield"]
        )

## Synchronous GET Data retrieval

As opposed to the POST method, the GET method requires the identifier itself and the price level. The remaining arguments are optional.

In [5]:
# Request bond PY with sync GET
py_sync_get_response = request_py_calculation_sync_by_id(
            id=identifier,
            level=price_level,
            curve_type="GVT",
            pricing_date="2025-01-01",
            currency="USD"
        )

## Asynchronous POST Data retrieval

The main difference between the Sync and Async approach is that Async methods allow execution of code on the user's side while waiting for a response. Once execution is complete, the user can 'collect' the results whenever they need them.

So this approach has 2 steps:
1. Provide input data to execution method
2. Use the request_id received in step 1 to retrieve the result

In [6]:
# Request bond PY with async post
py_async_post_response = request_py_calculation_async(
            global_settings=global_settings,
            input=input,
            keywords=["yield"]
        )

attempt = 1

while attempt < 10:

    from lseg_analytics.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_post_results_response = get_result(request_id_parameter=py_async_post_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + py_async_post_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{py_async_post_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

[ERROR]	[2025-09-18 08:58:30,435]	[MainThread]	[lseg_analytics.yield_book_rest]	[_functions.py:8525]	 Error get_result.
[ERROR]	[2025-09-18 08:58:30,436]	[MainThread]	[lseg_analytics]	[exceptions.py:78]	 Exception: Resource not found: code=404 {'meta': {'requestId': 'R-60873', 'status': 'RUNNING', 'responseType': 'PY_CALC', 'message': 'The result is not ready yet for requestID:R-60873', 'interval': 0.1}}
Attempt 1 resulted in error retrieving results from:R-60873


## Asynchronous GET Data retrieval

Difference between Sync and Async can be found in description above.

As Async execution is in queue-form it's not necessarily instantly finished. It can take time between submission of request and execution, and results are not in real-time.

Hence we present also the form of iterative results-request repetition to fetch the results as soon as they are ready.

**NOTE:** Alternative to this is wrapping the request in the job, and then using a method to check the job status. See the fundamentals notebook for information on jobs.

In [7]:
# Request bond PY with async get
py_async_get_response = request_py_calculation_async_by_id(
            id=identifier,
            level=price_level,
            curve_type="GVT",
            pricing_date="2025-01-01",
            currency="USD"
        )

# Due to async nature, code Will perform the fetch 10 times, as result is not always ready instantly, with 3 second lapse between attempts
attempt = 1

while attempt < 10:

    from lseg_analytics.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_get_results_response = get_result(request_id_parameter=py_async_get_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + py_async_get_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{py_async_get_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

[ERROR]	[2025-09-18 08:58:33,796]	[MainThread]	[lseg_analytics.yield_book_rest]	[_functions.py:8525]	 Error get_result.
[ERROR]	[2025-09-18 08:58:33,797]	[MainThread]	[lseg_analytics]	[exceptions.py:78]	 Exception: Resource not found: code=404 {'meta': {'requestId': 'R-60874', 'status': 'RUNNING', 'responseType': 'PY_CALC', 'message': 'The result is not ready yet for requestID:R-60874', 'interval': 0.1}}
Attempt 1 resulted in error retrieving results from:R-60874


## Display results

Results data is structured in JSON format so it can be easily displayed using print function.

### Synchronous POST results

In [8]:
# Print results in json format
print(js.dumps(py_sync_post_response, indent=4))

{
    "meta": {
        "status": "DONE",
        "requestId": "R-60871",
        "timeStamp": "2025-09-18T04:58:29Z",
        "responseType": "PY_CALC",
        "resultsStatus": "ALL"
    },
    "results": [
        {
            "py": {
                "yield": 4.685514
            },
            "securityID": "31398GY8"
        }
    ]
}


### Synchronous GET results

In [9]:
# Print results in json format
print(js.dumps(py_sync_get_response, indent=4))

{
    "data": {
        "py": {
            "wal": 6.008479,
            "dv01": 0.04868421,
            "isin": "US31398GY867",
            "cusip": "31398GY86",
            "price": 99.0,
            "yield": 4.685514,
            "ticker": "FNMA10.10",
            "cdYield": 0.0,
            "pyLevel": "99",
            "zSpread": 18.471014,
            "duration": 4.913874,
            "ziSpread": 18.471014,
            "znSpread": 55.276749,
            "benchmark": "5 yr",
            "className": "ZA",
            "convexity": 0.3854,
            "curveDate": "2024-12-31",
            "curveType": "Govt",
            "fullPrice": 99.075,
            "modelCode": 2501,
            "prepayRate": 100.0,
            "prepayType": "VEC",
            "securityID": "31398GY8",
            "tsyCurveID": "USDo1231",
            "accruedDays": 6,
            "description": "FEDERAL NATIONAL MORTGAGE ASSOCIATION 2010-10 ZA",
            "grossSpread": 24.3846,
            "pricingDate": "2

### Asynchronous POST results

In [10]:
# Print results in json format
print(js.dumps(async_post_results_response, indent=4))

{
    "meta": {
        "status": "DONE",
        "requestId": "R-60873",
        "timeStamp": "2025-09-18T04:58:30Z",
        "responseType": "PY_CALC",
        "resultsStatus": "ALL"
    },
    "results": [
        {
            "py": {
                "yield": 4.685514
            },
            "securityID": "31398GY8"
        }
    ]
}


### Asynchronous GET results

In [11]:
# Print results in json format
print(js.dumps(async_get_results_response, indent=4))

{
    "data": {
        "py": {
            "wal": 6.008479,
            "dv01": 0.04868421,
            "isin": "US31398GY867",
            "cusip": "31398GY86",
            "price": 99.0,
            "yield": 4.685514,
            "ticker": "FNMA10.10",
            "cdYield": 0.0,
            "pyLevel": "99",
            "zSpread": 18.471014,
            "duration": 4.913874,
            "ziSpread": 18.471014,
            "znSpread": 55.276749,
            "benchmark": "5 yr",
            "className": "ZA",
            "convexity": 0.3854,
            "curveDate": "2024-12-31",
            "curveType": "Govt",
            "fullPrice": 99.075,
            "modelCode": 2501,
            "prepayRate": 100.0,
            "prepayType": "VEC",
            "securityID": "31398GY8",
            "tsyCurveID": "USDo1231",
            "accruedDays": 6,
            "description": "FEDERAL NATIONAL MORTGAGE ASSOCIATION 2010-10 ZA",
            "grossSpread": 24.3846,
            "pricingDate": "2